# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [112]:
# importar a bibliotecas relevantes

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [114]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [116]:
# Tratamento das informações
df = df.dropna()

# Converter variáveis booleanas para numéricas
df['posse_de_veiculo'] = df['posse_de_veiculo'].astype(int)
df['posse_de_imovel'] = df['posse_de_imovel'].astype(int)

#Removendo linhas duplicadas.
df = df.drop_duplicates()

#Removendo colunas irrelevantes
df.drop(columns=['Unnamed: 0','data_ref', 'id_cliente'], inplace=True)

#Resetando o index.
df = df.reset_index()

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12427 entries, 0 to 12426
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  12427 non-null  int64  
 1   sexo                   12427 non-null  object 
 2   posse_de_veiculo       12427 non-null  int32  
 3   posse_de_imovel        12427 non-null  int32  
 4   qtd_filhos             12427 non-null  int64  
 5   tipo_renda             12427 non-null  object 
 6   educacao               12427 non-null  object 
 7   estado_civil           12427 non-null  object 
 8   tipo_residencia        12427 non-null  object 
 9   idade                  12427 non-null  int64  
 10  tempo_emprego          12427 non-null  float64
 11  qt_pessoas_residencia  12427 non-null  float64
 12  renda                  12427 non-null  float64
dtypes: float64(3), int32(2), int64(3), object(5)
memory usage: 1.1+ MB


In [120]:
df.head()

,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,F,0,1,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,M,1,1,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,F,1,1,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,F,0,1,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,M,1,0,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


##### <span style="color:blue"> 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [122]:
# Criar a variável dependente
df["log_renda"] = np.log(df["renda"])  

In [124]:
#Regressão 1

y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) +C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia', df)
modelo = sm.OLS(y,X).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 11 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:18:40   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

##### <span style="color:blue"> 2. Remova a variável menos significante e analise:
 - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
- Observe os parâmetros e veja se algum se alterou muito.to.

In [126]:
#Regressão 2
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(tipo_renda) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia', df)
modelo = sm.OLS(y,X).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     484.9
Date:                Tue, 11 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:21:21   Log-Likelihood:                -13605.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12412   BIC:                         2.735e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

### <span style="color:green"> OBS: Algumas variáveis, como C(tipo_renda)[T.Bolsista] e C(tipo_renda)[T.Pensionista], possuem  𝑝>0.05, indicando que podem não ser muito relevantes para prever a variável dependente.

##### <span style="color:blue"> 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [138]:
#Regressão 3 com apenas as varáveis com o-value menor que %5
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + idade + tempo_emprego + qt_pessoas_residencia', df)

modelo = sm.OLS(y,X).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1097.
Date:                Tue, 11 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:55:40   Log-Likelihood:                -13674.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 7.20

### <span style="color:green"> Se o objetivo for um modelo mais interpretável e eficiente, a Regressão 2 parece ser a melhor opção. Por outro lado, se a prioridade for um ajuste um pouco mais preciso e houver a possibilidade de aprimorar a Regressão 3 eliminando variáveis irrelevantes, essa alternativa também pode ser considerada.